In [1]:
!pip install torch transformers

In [2]:
import pandas as pd
# Carregue a data que possui categorias
data = pd.read_csv('/content/gpt_sampled_dataset.csv')

data.head()

,uid,street_name,label,history
0,maramores-ondr,Maramores,2,O nome desta rua foi solicitado pela Associaçã...
1,apaúra-aéon,Apaúra,5,Apaúra é o nome de um lago situado no Estado d...
2,apaúra-aéon,Apaúra,6,Apaúra é o nome de um lago situado no Estado d...
3,amerício-oa(n,Amerício,7,O amerício ( nome dado em homenagem ao Contine...
4,amerício-oa(n,Amerício,10,O amerício ( nome dado em homenagem ao Contine...


In [3]:
# Mapeamento de categorias
categorias = {
    "pessoa": 1,
    "comunidades": 2,
    "data": 3,
    "animal": 4,
    "natureza": 5,
    "objeto": 6,
    "termo": 7,
    "indígena": 8,
    "arte": 9,
    "local": 10,
    "outro": 11,
    "desconhecido": 12
}

# Dicionário de mapeamento de números de label para categorias
categorias_invertido = {v: k for k, v in categorias.items()}

# Aplicar o mapeamento inverso para criar uma nova coluna "categoria"
data['categoria'] = data['label'].map(categorias_invertido)

data.head()


,uid,street_name,label,history,categoria
0,maramores-ondr,Maramores,2,O nome desta rua foi solicitado pela Associaçã...,comunidades
1,apaúra-aéon,Apaúra,5,Apaúra é o nome de um lago situado no Estado d...,natureza
2,apaúra-aéon,Apaúra,6,Apaúra é o nome de um lago situado no Estado d...,objeto
3,amerício-oa(n,Amerício,7,O amerício ( nome dado em homenagem ao Contine...,termo
4,amerício-oa(n,Amerício,10,O amerício ( nome dado em homenagem ao Contine...,local


In [4]:
# Pré-processamento da data (você pode adaptar isso conforme necessário)
data['text'] = data['street_name'] + ' ' + data['history']
data['text'] = data['text'].fillna('')


In [5]:
data.head()

,uid,street_name,label,history,categoria,text
0,maramores-ondr,Maramores,2,O nome desta rua foi solicitado pela Associaçã...,comunidades,Maramores O nome desta rua foi solicitado pela...
1,apaúra-aéon,Apaúra,5,Apaúra é o nome de um lago situado no Estado d...,natureza,Apaúra Apaúra é o nome de um lago situado no E...
2,apaúra-aéon,Apaúra,6,Apaúra é o nome de um lago situado no Estado d...,objeto,Apaúra Apaúra é o nome de um lago situado no E...
3,amerício-oa(n,Amerício,7,O amerício ( nome dado em homenagem ao Contine...,termo,Amerício O amerício ( nome dado em homenagem a...
4,amerício-oa(n,Amerício,10,O amerício ( nome dado em homenagem ao Contine...,local,Amerício O amerício ( nome dado em homenagem a...


Testar códigos

- Código 1

In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


In [ ]:
# Divida seus dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Carregue o tokenizador BERT pré-treinado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize seus dados
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converta suas categorias em tensores
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

# Carregue o modelo BERT pré-treinado para classificação de sequências
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(categorias))

# Defina o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treine o modelo
model.train()
optimizer.zero_grad()
outputs = model(**X_train_tokens, labels=y_train_tensor)
loss = outputs.loss
loss.backward()
optimizer.step()

# Avalie o modelo no conjunto de teste
model.eval()
with torch.no_grad():
    outputs = model(**X_test_tokens)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

# Mapeie os rótulos preditos de volta para as categorias cognitivas
predicted_categories = [list(categorias.keys())[list(categorias.values()).index(label)] for label in predicted_labels.tolist()]

# Calcule a acurácia
accuracy = accuracy_score(y_test_tensor.tolist(), predicted_labels.tolist())
print(f'Acurácia do modelo: {accuracy:.2f}')

# Exiba as categorias cognitivas previstas
print(predicted_categories)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- Código 2: coluna em string

In [12]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW


In [13]:
# Divida seus dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['categoria'], test_size=0.2, random_state=42)

# Carregue o tokenizador ALBERT pré-treinado
tokenizer = AutoTokenizer.from_pretrained('albert-base-v2')

# Tokenize seus dados
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converta suas categorias em tensores
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

# Carregue o modelo ALBERT pré-treinado para classificação de sequências
model = AutoModelForSequenceClassification.from_pretrained('albert-base-v2', num_labels=len(set(y_train)))

# Defina o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treine o modelo
model.train()
optimizer.zero_grad()
outputs = model(**X_train_tokens, labels=y_train_tensor)
loss = outputs.loss
loss.backward()
optimizer.step()

batch_size = 8  # Tamanho do lote menor

# Divida o conjunto de teste em lotes menores
num_batches = len(X_test_tokens) // batch_size

predicted_labels = []

model.eval()
with torch.no_grad():
    for i in range(num_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        batch_inputs = {key: val[start:end] for key, val in X_test_tokens.items()}

        outputs = model(**batch_inputs)
        logits = outputs.logits
        batch_predicted_labels = torch.argmax(logits, dim=1)
        predicted_labels.extend(batch_predicted_labels.tolist())

# Certifique-se de lidar com o último lote (que pode ser menor)
start = num_batches * batch_size
batch_inputs = {key: val[start:] for key, val in X_test_tokens.items()}
outputs = model(**batch_inputs)
logits = outputs.logits
batch_predicted_labels = torch.argmax(logits, dim=1)
predicted_labels.extend(batch_predicted_labels.tolist())

# Calcule a acurácia
accuracy = accuracy_score(y_test_tensor.tolist(), predicted_labels)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Exiba as categorias cognitivas previstas
print(predicted_labels)

ValueError: ignored

- Código3: Distil Bert

In [14]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW


In [ ]:

# Divida seus dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Carregue o tokenizador BERT pré-treinado
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize seus dados
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converta suas categorias em tensores
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

# Carregue o modelo BERT pré-treinado para classificação de sequências
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(categorias))

# Defina o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treine o modelo
model.train()
optimizer.zero_grad()
outputs = model(**X_train_tokens, labels=y_train_tensor)
loss = outputs.loss
loss.backward()
optimizer.step()

# Avalie o modelo no conjunto de teste
model.eval()
with torch.no_grad():
    outputs = model(**X_test_tokens)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=1)

# Mapeie os rótulos preditos de volta para as categorias cognitivas
predicted_categories = [list(categorias.keys())[list(categorias.values()).index(label)] for label in predicted_labels.tolist()]

# Calcule a acurácia
accuracy = accuracy_score(y_test_tensor.tolist(), predicted_labels.tolist())
print(f'Acurácia do modelo: {accuracy:.2f}')

# Exiba as categorias cognitivas previstas
print(predicted_categories)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- Código 4: Distil Bert with data train divide

In [6]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW


In [ ]:
# Divida seus dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Carregue o tokenizador BERT pré-treinado
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize seus dados
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converta suas categorias em tensores
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

# Carregue o modelo BERT pré-treinado para classificação de sequências
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(categorias))

# Defina o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treine o modelo
model.train()
optimizer.zero_grad()
outputs = model(**X_train_tokens, labels=y_train_tensor)
loss = outputs.loss
loss.backward()
optimizer.step()

batch_size = 32  # Tamanho do lote

# Divida o conjunto de teste em lotes menores
num_batches = len(X_test_tokens) // batch_size

predicted_labels = []

model.eval()
with torch.no_grad():
    for i in range(num_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        batch_inputs = {key: val[start:end] for key, val in X_test_tokens.items()}

        outputs = model(**batch_inputs)
        logits = outputs.logits
        batch_predicted_labels = torch.argmax(logits, dim=1)
        predicted_labels.extend(batch_predicted_labels.tolist())

# Certifique-se de lidar com o último lote (que pode ser menor)
start = num_batches * batch_size
batch_inputs = {key: val[start:] for key, val in X_test_tokens.items()}
outputs = model(**batch_inputs)
logits = outputs.logits
batch_predicted_labels = torch.argmax(logits, dim=1)
predicted_labels.extend(batch_predicted_labels.tolist())

# Calcule a acurácia
accuracy = accuracy_score(y_test_tensor.tolist(), predicted_labels)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Exiba as categorias cognitivas previstas
print(predicted_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- Código 5: Albert

In [5]:
!pip install sentencepiece

In [6]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AlbertTokenizer, AlbertForSequenceClassification, AdamW

In [ ]:
# Divida seus dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Carregue o tokenizador ALBERT pré-treinado
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Tokenize seus dados
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converta suas categorias em tensores
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

# Carregue o modelo ALBERT pré-treinado para classificação de sequências
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=len(categorias))

# Defina o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treine o modelo
model.train()
optimizer.zero_grad()
outputs = model(**X_train_tokens, labels=y_train_tensor)
loss = outputs.loss
loss.backward()
optimizer.step()

batch_size = 32  # Tamanho do lote

# Divida o conjunto de teste em lotes menores
num_batches = len(X_test_tokens) // batch_size

predicted_labels = []

model.eval()
with torch.no_grad():
    for i in range(num_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        batch_inputs = {key: val[start:end] for key, val in X_test_tokens.items()}

        outputs = model(**batch_inputs)
        logits = outputs.logits
        batch_predicted_labels = torch.argmax(logits, dim=1)
        predicted_labels.extend(batch_predicted_labels.tolist())

# Certifique-se de lidar com o último lote (que pode ser menor)
start = num_batches * batch_size
batch_inputs = {key: val[start:] for key, val in X_test_tokens.items()}
outputs = model(**batch_inputs)
logits = outputs.logits
batch_predicted_labels = torch.argmax(logits, dim=1)
predicted_labels.extend(batch_predicted_labels.tolist())

# Calcule a acurácia
accuracy = accuracy_score(y_test_tensor.tolist(), predicted_labels)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Exiba as categorias cognitivas previstas
print(predicted_labels)


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- Código 6: Código utilizando transformers com a capacidade de exigência do pc reduzida

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW


In [ ]:
# Divida seus dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Carregue o tokenizador ALBERT pré-treinado
tokenizer = AutoTokenizer.from_pretrained('albert-base-v2')

# Tokenize seus dados
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converta suas categorias em tensores
y_train_tensor = torch.tensor(y_train.tolist())
y_test_tensor = torch.tensor(y_test.tolist())

# Carregue o modelo ALBERT pré-treinado para classificação de sequências
model = AutoModelForSequenceClassification.from_pretrained('albert-base-v2', num_labels=len(categorias))

# Defina o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treine o modelo
model.train()
optimizer.zero_grad()
outputs = model(**X_train_tokens, labels=y_train_tensor)
loss = outputs.loss
loss.backward()
optimizer.step()

batch_size = 2  # Tamanho do lote menor

# Divida o conjunto de teste em lotes menores
num_batches = len(X_test_tokens) // batch_size

predicted_labels = []

model.eval()
with torch.no_grad():
    for i in range(num_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        batch_inputs = {key: val[start:end] for key, val in X_test_tokens.items()}

        outputs = model(**batch_inputs)
        logits = outputs.logits
        batch_predicted_labels = torch.argmax(logits, dim=1)
        predicted_labels.extend(batch_predicted_labels.tolist())

# Certifique-se de lidar com o último lote (que pode ser menor)
start = num_batches * batch_size
batch_inputs = {key: val[start:] for key, val in X_test_tokens.items()}
outputs = model(**batch_inputs)
logits = outputs.logits
batch_predicted_labels = torch.argmax(logits, dim=1)
predicted_labels.extend(batch_predicted_labels.tolist())

# Calcule a acurácia
accuracy = accuracy_score(y_test_tensor.tolist(), predicted_labels)
print(f'Acurácia do modelo: {accuracy:.2f}')

# Exiba as categorias cognitivas previstas
print(predicted_labels)



Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


- Código 7: Rede Neural

In [6]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, AutoModel
import torch.nn as nn


In [ ]:
# Divida seus dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)

# Carregue o tokenizador ALBERT pré-treinado
tokenizer = AutoTokenizer.from_pretrained('albert-base-v2')

# Tokenize seus dados
X_train_tokens = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=512,
                           return_attention_mask=True)
X_test_tokens = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='pt', max_length=512,
                          return_attention_mask=True)

# Converta suas categorias em tensores
y_train_tensor = torch.tensor(y_train.tolist(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.tolist(), dtype=torch.long)

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.embedding = AutoModel.from_pretrained('albert-base-v2')  # Use o modelo ALBERT como camada de embedding
        self.conv1 = nn.Conv1d(128, 256, kernel_size=5)  # Camada de convolução
        self.fc1 = nn.Linear(256 * (512 - 4), 128)  # Camada totalmente conectada com tamanho ajustado
        self.fc2 = nn.Linear(128, num_classes)  # Camada totalmente conectada para saída final

    def forward(self, input_ids, attention_mask):
        outputs = self.embedding(input_ids, attention_mask=attention_mask)
        x = outputs.last_hidden_state  # Pega a saída da camada de embedding do ALBERT
        x = x.permute(0, 2, 1)  # Permutar dimensões para a convolução
        x = self.conv1(x)
        x = torch.max(x, 2)[0]  # Pooling máximo
        x = x.view(x.size(0), -1)  # Redimensionar para o formato adequado
        x = self.fc1(x)
        x = self.fc2(x)  # Camada final
        return x


# Crie o modelo CNN
model = SimpleCNN(len(set(y_train)))

# Defina o otimizador
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treine o modelo
model.train()
optimizer.zero_grad()
outputs = model(X_train_tokens['input_ids'], X_train_tokens['attention_mask'])
loss_function = nn.CrossEntropyLoss()
loss = loss_function(outputs, y_train_tensor)
loss.backward()
optimizer.step()

# Avalie o modelo no conjunto de teste
model.eval()
with torch.no_grad():
    outputs = model(X_test_tokens['input_ids'], X_test_tokens['attention_mask'])
    predicted_labels = torch.argmax(outputs, dim=1)

# Calcule a acurácia
accuracy = accuracy_score(y_test_tensor.tolist(), predicted_labels.tolist())
print(f'Acurácia do modelo: {accuracy:.2f}')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
